# Preparation of barcode to mutant variant assignment

The CbbM combinatorial and saturational libraries were digested separately with single-cutter restriction enzymes. After some purification steps, DNA was sent to NGI in Uppsala for PacBio library preparation and sequencing.

## Process PacBio sequencing results to assign barcodes to mutant variants

### Preprocessing

PacBio sequences were mapped using minimap2 v2.28 (r1209) and its CCS-specific preset to a 1,675 bp long template spanning the promoter region (Ptrc), coding sequence of cbbM(I) and a small region up- and downstream of the N20 barcode. The resulting sam-file was filtered for aligned reads with a mapping quality above 5 (MAPQ > 5) using the samtools v1.13 view command.

`./minimap2/minimap2 -ax map-hifi ref_cbbM_onlyInsert.fa m84045_240523_204011_s1.fastq > alignment.sam`

`samtools view alignment.sam -q 5 > alignment_onlyMapped.sam`

minimap2 gave the following output: 


[M::mm_idx_gen::0.001*3.24] collected minimizers
[M::mm_idx_gen::0.004*3.03] sorted minimizers
[M::main::0.004*3.01] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.004*2.96] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.004*2.93] distinct minimizers: 161 (100.00% are singletons); average occurrences: 1.000; average spacing: 10.404; total length: 1675
[M::worker_pipeline::11.694*2.95] mapped 74228 sequences
[M::worker_pipeline::21.712*3.10] mapped 74275 sequences
[M::worker_pipeline::32.850*3.17] mapped 74267 sequences
[M::worker_pipeline::52.027*3.17] mapped 74283 sequences
[M::worker_pipeline::67.171*2.58] mapped 74273 sequences
[M::worker_pipeline::88.953*2.70] mapped 74282 sequences
[M::worker_pipeline::95.307*2.76] mapped 76924 sequences
[M::worker_pipeline::113.167*2.83] mapped 80983 sequences
[M::worker_pipeline::140.630*2.37] mapped 76552 sequences
[M::worker_pipeline::155.093*2.37] mapped 75791 sequences
[M::worker_pipeline::172.260*2.29] mapped 84966 sequences
[M::worker_pipeline::201.381*2.14] mapped 76781 sequences
[M::worker_pipeline::216.760*2.16] mapped 75404 sequences
[M::worker_pipeline::246.265*2.07] mapped 75751 sequences
[M::worker_pipeline::260.820*2.13] mapped 74410 sequences
[M::worker_pipeline::290.399*2.01] mapped 74258 sequences
[M::worker_pipeline::303.445*2.06] mapped 75417 sequences
[M::worker_pipeline::325.591*2.02] mapped 80280 sequences
[M::worker_pipeline::348.793*1.97] mapped 91360 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: ./minimap2/minimap2 -ax map-hifi ref_cbbM_onlyInsert.fa m84045_240523_204011_s1.fastq
[M::main] Real time: 348.794 sec; CPU: 685.999 sec; Peak RSS: 1.933 GB

Aligned reads were extracted from the resulting sam file and converted into a fasta format with a python 3.9.7 script (sam2fasta.py).

In [ ]:
# sam2fasta.py
with open("alignment_onlyMapped.sam") as sam_file:
    with open("aligned_seq.fasta", "w") as fasta_file:
        for line in sam_file:
            if line[0] == "@":
                continue
            else:
                seq_name = line.split("\t")[0]
                sequence = line.split("\t")[9]
                fasta_file.write(">" + seq_name + "\n" + sequence + "\n")

To extract coding sequences and barcodes from this file, cutadapt v3.5 was run using two different sets of paired adapters corresponding to the sequences exactly up- and downstream of the cbbM coding sequence and the N20 barcode, respectively. Untrimmed reads were discarded. 

`cutadapt -a ACTGAAACATCTTAATCATGCTAAGGAGGTTTTCTA...TCTAGAATCGCCGAAAGTAATTCAACTCCATTAA --discard-untrimmed -o CDS_trim.fasta aligned_seq.fasta > cutadapt_CDSreport.txt`

`cutadapt -a TCTAGAATCGCCGAAAGTAATTCAACTCCATTAA...TCTAGATGCTTACTAGTTACCGCGGCCAGGCAT --discard-untrimmed -o barcode_trim.fasta aligned_seq.fasta 2> cutadapt_barcode_report.txt`

A set of python 3.9.7 scripts were used to assign barcodes to coding sequences and translate the latter to protein sequences (extract_from_barcodeFasta.py, extract_from_CDS_Fasta.py). In the same step, the set of extracted barcodes was separated into three subsets according to the number of their occurrences. Furthermore, coding sequences translated into protein products shorter than 400 amino acids were discarded. 

In [1]:
# originally: extract_from_barcodeFasta.py
# Open file created by cutadapt, extract barcodes, discard barcodes < 15 nt or > 20 nt
# save file with barcode, seqname, number of occurrences
barcode_dict = {}

with open("barcode_trim.fasta") as barcode_file:
    for line in barcode_file:
        if line[0] == ">":
            seq_name = line.strip("\n")[1:]
        else:
            barcode = line.strip("\n")
            if len(barcode) > 20 or len(barcode) < 15:
                continue
            else:
                if barcode not in barcode_dict.keys():
                    barcode_dict[barcode] = [1,[seq_name]]
                else:
                    barcode_dict[barcode][0] += 1
                    barcode_dict[barcode][1].append(seq_name)

with open("interm_file/barcode_occurrences_seqname.tsv", "w") as barcode_file:
    barcode_file.write("barcode\toccurrences\tseqname\n")
    for barcode in barcode_dict.keys():
        for seqname in barcode_dict[barcode][1]:
            barcode_file.write(str(barcode) + "\t" + str(barcode_dict[barcode][0]) + "\t" + seqname + "\n")

In [4]:
# originally: extract_from_CDS_Fasta.py
# separate barcodes according to number of occurrences into 3 sets (save RAM later)
# assign CDS to barcodes according to shared seqname (from cutadapt output file)
# discard CDS shorter than 300 aa (unlikely to be relevant for us)
from Bio.Seq import Seq

def pad_seq(sequence):
    """ Pad sequence to multiple of 3 with N """ # from https://stackoverflow.com/questions/53894575/how-can-i-fix-this-error-biopythonwarning-partial-codon-lensequence-not-a
    remainder = len(sequence) % 3
    return sequence if remainder == 0 else sequence + Seq('N' * (3 - remainder))

seqnames_of_interest_occ1 = {}
seqnames_of_interest_occ10 = {}
seqnames_of_interest_rest = {}

counter = 0
with open("interm_file/barcode_occurrences_seqname.tsv") as barcode_table:
    line_one = True
    for line in barcode_table:
        if line_one:
            line_one = False
            continue
        occurrences = int(line.strip("\n").split("\t")[1])
        barcode = line.split("\t")[0]
        seq = line.strip("\n").split("\t")[2]
        if occurrences == 1:
            seqnames_of_interest_occ1[seq] = barcode
        if occurrences > 1 and occurrences < 11:
            seqnames_of_interest_occ10[seq] = barcode
        if occurrences > 10:
            seqnames_of_interest_rest[seq] = barcode

print("Went through first file")

with open("interm_file/barcode_CDS_1occ.tsv", "w") as f:
    f.write("")
with open("interm_file/barcode_CDS_10occ.tsv", "w") as f:
    f.write("")
with open("interm_file/barcode_CDS_rest.tsv", "w") as f:
    f.write("")
counter = 0
with open("CDS_trim.fasta") as CDS_file:
        for line in CDS_file:
            counter += 1
            if counter % 50000 == 0:
                print("counter is " + str(counter))
            if line[0] == ">":
                seq_name = line.strip("\n")[1:]
            else:                
                my_dna = Seq(pad_seq(line.strip("\n")))
                my_aa = str(my_dna.translate()).split("*")[0]
                if len(my_aa) < 300:
                    continue
                if seq_name in seqnames_of_interest_occ1.keys():
                    with open("interm_file/barcode_CDS_1occ.tsv", "a") as f:
                        f.write(seqnames_of_interest_occ1[seq_name] + "\t" + my_aa + "\n")
                if seq_name in seqnames_of_interest_occ10.keys():
                    with open("interm_file/barcode_CDS_10occ.tsv", "a") as f:
                        f.write(seqnames_of_interest_occ10[seq_name] + "\t" + my_aa + "\n")
                if seq_name in seqnames_of_interest_rest.keys():
                    with open("interm_file/barcode_CDS_rest.tsv", "a") as f:
                        f.write(seqnames_of_interest_rest[seq_name] + "\t" + my_aa + "\n")

Went through first file
counter is 50000
counter is 100000
counter is 150000
counter is 200000
counter is 250000
counter is 300000
counter is 350000
counter is 400000
counter is 450000
counter is 500000
counter is 550000
counter is 600000
counter is 650000
counter is 700000
counter is 750000
counter is 800000
counter is 850000
counter is 900000
counter is 950000
counter is 1000000
counter is 1050000
counter is 1100000
counter is 1150000
counter is 1200000
counter is 1250000
counter is 1300000
counter is 1350000
counter is 1400000
counter is 1450000
counter is 1500000
counter is 1550000
counter is 1600000
counter is 1650000
counter is 1700000
counter is 1750000
counter is 1800000
counter is 1850000
counter is 1900000
counter is 1950000
counter is 2000000
counter is 2050000
counter is 2100000
counter is 2150000
counter is 2200000
counter is 2250000
counter is 2300000
counter is 2350000
counter is 2400000
counter is 2450000
counter is 2500000
counter is 2550000


In [5]:
# to clear RAM: delete dictionaries
del seqnames_of_interest_occ1
del seqnames_of_interest_occ10
del seqnames_of_interest_rest

For barcodes occurring more than once, the assigned coding sequences were compared and the sequence occurring most frequently was used as consensus sequence. In case there were ties, these barcodes were omitted from future analysis (find_consensus_sequences.py and find_counsensus_sequences_over100.py). 

In [20]:
# originally: find_consensus_sequences.py and find_counsensus_sequences_over100.py
# for barcodes that occur between 2 and 10 times, find consensus sequence
# assumes that longer CDS is more correct than shorter CDS (assumption that early stop codons are sequencing mistake)

barcode_dict = {}

with open("interm_file/barcode_CDS_10occ.tsv") as barc_file:
    for line in barc_file:
        barcode = line.split("\t")[0]
        CDS = line.strip("\n").split("\t")[1]
        if barcode not in barcode_dict.keys():
            barcode_dict[barcode] = [CDS]
        else:
            barcode_dict[barcode].append(CDS)

with open("interm_file/barcode_CDS_rest.tsv") as barc_file:
    for line in barc_file:
        barcode = line.split("\t")[0]
        CDS = line.strip("\n").split("\t")[1]
        if barcode not in barcode_dict.keys():
            barcode_dict[barcode] = [CDS]
        else:
            barcode_dict[barcode].append(CDS)
            
print("Number of barcodes to be checked: " + str(len(barcode_dict.keys())))

with open("interm_file/barcode_CDS_consensus.tsv", "w") as f:
    f.write("")
with open("interm_file/barcode_CDS_ties.tsv", "w") as f:
    f.write("")
with_ties = 0
no_long_CDS = 0
for barcode in barcode_dict.keys():
    CDS_dict = {}
    max_count = 1
    curr_count = 0
    most_freq_CDS = []
    for CDS in barcode_dict[barcode]:
        if CDS not in CDS_dict.keys():
            CDS_dict[CDS] = 1
            curr_count = 1
            if curr_count == max_count and CDS not in most_freq_CDS:
                most_freq_CDS.append(CDS)
        else:
            CDS_dict[CDS] += 1
            curr_count = CDS_dict[CDS]
            if curr_count > max_count:
                max_count = curr_count
                most_freq_CDS = [CDS]
            elif curr_count == max_count and CDS not in most_freq_CDS:
                most_freq_CDS.append(CDS)
    if not most_freq_CDS:
        no_long_CDS += 1
        continue
    elif len(most_freq_CDS) == 1:
        with open("interm_file/barcode_CDS_consensus.tsv", "a") as f:
            f.write(str(barcode) + "\t" + most_freq_CDS[0] + "\n")
    else:
        max_len_CDS = 0
        longest_CDS = []
        for CDS in most_freq_CDS:
            curr_len_CDS = len(CDS)
            if curr_len_CDS > max_len_CDS:
                max_len_CDS = curr_len_CDS
                longest_CDS = [CDS]
            if curr_len_CDS == max_len_CDS:
                longest_CDS.append(CDS)
        if len(longest_CDS) > 1:
            with_ties += 1
            with open("interm_file/barcode_CDS_ties.tsv", "a") as f:
                for CDS in most_freq_CDS:
                    f.write(str(barcode) + "\t" + CDS + "\n")
        else:
            with open("interm_file/barcode_CDS_consensus.tsv", "a") as f:
                f.write(str(barcode) + "\t" + longest_CDS[0] + "\n")
            
                
print(str(with_ties) + " barcodes with ties and " + str(no_long_CDS) + " barcodes with no most frequent CDS")

Number of barcodes to be checked: 177406
9601 barcodes with ties and 0 barcodes with no most frequent CDS


Identify barcodes which are to be expected

In [21]:
# originally: create_table_expected_mutations.py
# creates table of barcodes which are to be expected in library
aa_dict = "ARNDCEQGHILKMFPSTWYV"

dict_expected_mutations = {}
with open("input/expected_saturational_mutations.tsv") as f: 
    for line in f:
        aa_pos = line.strip("\n")
        WT_aa = aa_pos[0]
        for aa in aa_dict:
            if aa == WT_aa:
                continue
            else:
                dict_expected_mutations[aa_pos + aa] = 0
                
with open("saturational_mutations.tsv", "w") as f:
    for mut in dict_expected_mutations.keys():
        f.write(str(mut) + "\n")
print(len(dict_expected_mutations.keys()))

combinatorial_dict = {}
with open("input/expected_combinatorial_mutations.tsv") as f:
    for line in f: 
        aa_pos = line.split("\t")[0]
        possible_aa = line.strip("\n").split("\t")[1].split(",")
        combinatorial_dict[aa_pos] = possible_aa

dict_expected_combi = {}
list_combi = ["H127", "M140", "K247", "H251", "V323", "Q392", "S432"]
H_combis = []
for H_aa in combinatorial_dict["H141"]:
    if H_aa == "H":
        H_combis.append("")
    else:
        H_combis.append("H141" + H_aa + ",")
M_combis = []
for M_aa in combinatorial_dict["M154"]:
    if M_aa == "M": 
        for H in H_combis:
            M_combis.append(H)    
    else:
        for H in H_combis:
            M_combis.append(H + "M154" + M_aa + ",")
K_combis = []
for K_aa in combinatorial_dict["K261"]:
    if K_aa == "K": 
        for M in M_combis:        
            K_combis.append(M)
    else: 
        for M in M_combis:        
            K_combis.append(M + "K261" + K_aa + ",")
H_combis = []
for H_aa in combinatorial_dict["H265"]:
    if H_aa == "H": 
        for K in K_combis:
            H_combis.append(K)
    else:
        for K in K_combis:
            H_combis.append(K +"H265" + H_aa + ",")
V_combis = []
for V_aa in combinatorial_dict["V337"]:
    if V_aa == "V":
        for H in H_combis:
            V_combis.append(H)
    else:
        for H in H_combis:
            V_combis.append(H +"V337" + V_aa + ",")
Q_combis = []
for Q_aa in combinatorial_dict["Q406"]:
    if Q_aa == "Q": 
        for V in V_combis:
            Q_combis.append(V)
    else:
        for V in V_combis:
            Q_combis.append(V + "Q406" + Q_aa + ",")
S_combis = []
for S_aa in combinatorial_dict["S446"]:
    if S_aa == "S":
        for Q in Q_combis:
            S_combis.append(Q)
    else:
        for Q in Q_combis:
            S_combis.append(Q + "S446" + S_aa + ",")
for combi in S_combis:
    if combi == "":
        dict_expected_combi["WT,"] = 1
        continue
    dict_expected_combi[combi] = 1
                                
with open("combinatorial_mutations.tsv", "w") as f:
    for mut in dict_expected_combi.keys():
        f.write(str(mut)[:-1] + "\n")
print(len(dict_expected_combi.keys()))

1805
16200


In a final step, all barcodes were combined into one file, amino acid exchanges compared to the CbbM(I) sequence were determined and the combination of barcodes and assigned mutations were formatted to be compatible with downstream applications (process_all_sequences.py).

In [3]:
# originally: process_all_sequences.py
# combines all barcodes and mutations in one file
# compares to list of expected mutant variants - saves this into separate file
barc_dict = {}

total = 0
with open("interm_file/barcode_CDS_1occ.tsv") as f:
    for line in f:
        total += 1
        barcode = line.split("\t")[0]
        seq = line.strip("\n").split("\t")[1]
        barc_dict[barcode] = seq
print("after 1 occ barcodes\ntotal: " + str(total))

with open("interm_file/barcode_CDS_consensus.tsv") as f:
    for line in f:
        total += 1
        barcode = line.split("\t")[0]
        seq = line.strip("\n").split("\t")[1]
        barc_dict[barcode] = seq
print("With consensus sequences\ntotal: " + str(total))

with open("output/all_barcodes_complete_CDS.tsv", "w") as f:
    for barcode in barc_dict:
        f.write(barcode + "\t" + barc_dict[barcode] + "\n")

expected_mut = {}
with open("saturational_mutations.tsv") as f:
    for line in f:
        expected_mut[line.strip("\n")] = 1
with open("combinatorial_mutations.tsv") as f:
    for line in f:
        expected_mut[line.strip("\n")] = 1
print("Expected mutant variants: " + str(len(expected_mut.keys())))
found_expected_mut_variants = 0
found_exp_mut_variants = {}
with open("output/expected_mutations_barcodes.tsv", "w") as f2:
    f2.write("")
original_seq = "MWSHPQFEKGSGSGSDQSNRYANLNLKEEDLIKNGKHLLVAYKLIPAKGHGFLEVAAHVAAESSTGTNVEVSTTDDFTRGVDALVYEIDETAFGDDPVKGGGLFKVAYPVELFDPNLTDGTYNISHMWSLILGNNQGMGDHQGLRMLDFLVPEMMVRKFDGPSANISNLWKVLGRPETDGGYIAGTIIKPKLGLRPEPFAKACYDFWLGGDFIKNDEPQANQPFCPMEVVMPKVAEAMDRAQQATGQAKLFSANITADYYKEMIHRGDFVLETFAKYNSASHVAFLVDGFVTGPAGVTTARREFPDTFLHFHRAGHGAVTSYKSPMGMDPLCYMKLVRLMGASGMHTGTMGYGKMEGHGKETVLAYMLERDECQGPYFYQKWYGMKATTPIISGGMNALRLPGFFQNLGHGNVINTCGGGAFGHIDSPAAGGISLRQAYDCWKSGSDPIEYAKTHKEFARAFESFPKDGDKLFAGWREKLGVHK"
mutation_to_barc = {}
with open("output/all_barcodes_mutations.tsv", "w") as f:
    for barcode in barc_dict:
        mutation_code = ""
        seq = barc_dict[barcode]
        if len(seq) > len(original_seq):
            for index in range(len(seq)):
                if index == len(original_seq):
                    mutation_code += ("X" + str(index) + seq[index] + ",")
                    break
                if original_seq[index] == seq[index]:
                    continue
                else:
                    mutation_code += (original_seq[index] + str(index+1) + seq[index] + ",")
        else:
            for index in range(len(original_seq)):
                if index == len(seq):
                    mutation_code += "STOP" + str(index) + ","
                    break
                if original_seq[index] == seq[index]:
                    continue
                else:
                    mutation_code += (original_seq[index] + str(index+1) + seq[index] + ",")
        if mutation_code == "":
            mutation_code = "WT,"
        f.write(barcode + "\t" + mutation_code[:-1] + "\n")
        if mutation_code[:-1] not in mutation_to_barc.keys():
            mutation_to_barc[mutation_code[:-1]] = [barcode]
        else:
            mutation_to_barc[mutation_code[:-1]].append(barcode)
        with open("output/expected_mutations_barcodes.tsv", "a") as f2:
            if mutation_code[:-1] in expected_mut.keys():
                found_expected_mut_variants += 1
                found_exp_mut_variants[mutation_code[:-1]] = 1
                f2.write(barcode + "\t" + mutation_code[:-1] + "\n")
                
mutations_covered = []
length_constraints = 0
with open("output/unfiltered_barcodes.fasta", "w") as f:
    for mut in mutation_to_barc.keys():
        if len(str(mut)) > 190: # featureCounts upper limit is 199 characters
            length_constraints += 1
            continue
        if mut in expected_mut.keys():
            for i in range(len(mutation_to_barc[mut])):
                f.write(">" + str(mut) + "|" + str(i+1) + "\n" + mutation_to_barc[mut][i] + "\n")
        else:
            if len(mutation_to_barc[mut]) > 1:
                mutations_covered += [mut]
                for i in range(len(mutation_to_barc[mut])):
                    f.write(">" + str(mut) + "|" + str(i+1) + "\n" + mutation_to_barc[mut][i] + "\n")
print(str(length_constraints) + " mutant variants excluded due to length constraints")
print("Mutant variants not expected, covered by more than one barcode: " + str(len(mutations_covered)) + ", for instance: " + str(mutations_covered[1:10]))
                    
print("Found barcodes for expected mutant variants: " + str(found_expected_mut_variants) + ", for number mutant variants: " + str(len(found_exp_mut_variants.keys())))

not_found = []
for mut in expected_mut.keys():
    if mut not in found_exp_mut_variants.keys():
        not_found.append(mut)

print("Variants not found: " + str(not_found))

after 1 occ barcodes
total: 67793
With consensus sequences
total: 235598
Expected mutant variants: 17983
3518 mutant variants excluded due to length constraints
Mutant variants not expected, covered by more than one barcode: 1298, for instance: ['G395V,STOP395', 'H141L,M154S,K261E,H265A,V337A,F405S,Q406K,N407T,STOP407', 'A83V,H141L,M154S,K261E,H265R,V337S,Q406D,S446I', 'K261F,Q406P,N407E,G409R,H410A,G411R,N412E,V413R,I414N,STOP414', 'Q406P,N407E,G409R,H410A,G411R,N412E,V413R,I414N,STOP414', 'STOP360', 'M326R,T347C', 'M154E,G395V,STOP395', 'A295G,V297G,T298D,T299H,A300C,R301T,R302A,E303R,F304V,D306R,T307Y,F308L,H310A,F311L,H312S,R313S,A314R,H316T,G317W,A318G,V319S,T320H,S321I,Y322L,STOP322']
Found barcodes for expected mutant variants: 205150, for number mutant variants: 17839
Variants not found: ['H141V,M154E,K261A', 'H141L,M154S,K261E,H265K', 'H141L,K261F,H265R', 'H141L,M154A,K261F,H265E', 'M154K,H265K,V337A', 'M154K,K261D,H265K,V337A', 'H141L,K261F,H265K,V337A', 'H141V,M154S,K261D,H2

## Identify barcode subset for Nextflow analysis

Run Nextflow once with unfiltered_barcodes.fasta, then select
Also check for other mut variants than WT if they really need that many barcodes (25 should be alright?).
Approx. 500 barcodes for WT, of which many are detected less than x times.
Also, for mapping: reduce library to barcodes which were identified in all_counts.tsv (mapping of Illumina data from first large library cultivation)
For further analyses, mutant variants which were not part of the originally designed library were only kept if they were represented by at least two barcodes

In [12]:
# Check all_counts.tsv for which barcodes were identified by Illumina sequencing in Synechocystis library
# discard al barcodes which were not found in the Synechocystis library
# "WT" has many many many barcodes (>100) - only keep 25 barcodes which were scored highest
# if mutant variant is not in the set of expected barcodes: only keep variant in library if more than one
# barcode is assigned to mutant variant and detected by Illumina sequencing

all_counts_mutations = {} # just collecting which mutant variants are in file
all_counts_mut_with_counts = {} # dictionary with barcodes assigned to each mutant variant
all_barcodes_with_counts = {} # all barcodes and how often they were counted
number_muts_with_redundancy = 0
first_line = True
with open("input/all_counts.tsv") as f: 
    for line in f:
        if first_line:
            first_line = False
            continue
        split_line = line.strip("\n").split("\t")
        barcode_name = split_line[0]
        mutant_variant = split_line[1]
        all_counts_mutations[mutant_variant] = 1
        col_sum = 0
        for col in split_line[2:]:
            col_sum += int(col)
        if col_sum > 30:
            if mutant_variant not in all_counts_mut_with_counts.keys():
                all_counts_mut_with_counts[mutant_variant] = [barcode_name]
            else:
                all_counts_mut_with_counts[mutant_variant].append(barcode_name)
            if barcode_name not in all_barcodes_with_counts.keys():
                all_barcodes_with_counts[barcode_name] = col_sum
            else:
                all_barcodes_with_counts[barcode_name] += col_sum
                
                
numbers = []    
mutations = []
for mut in all_counts_mut_with_counts.keys():
    numbers.append(len(all_counts_mut_with_counts[mut]))
    mutations.append(str(mut))
    if len(all_counts_mut_with_counts[mut]) > 1:
        number_muts_with_redundancy += 1
import pandas as pd
df = pd.DataFrame(mutations, numbers)
df.to_csv("numbers.csv")
        
print("All different mutant variants in all_counts.tsv (regardless of if counts): " + str(len(all_counts_mutations.keys())))
print("Mutant variants with counts > 30: " + str(len(all_counts_mut_with_counts.keys())))
print("Number mutant variants with redundancy: " + str(number_muts_with_redundancy))
print("All barcodes with counts > 30: " + str(len(all_barcodes_with_counts.keys())))

expected_mutations = {}
with open("combinatorial_mutations.tsv") as f:
    for line in f:
        expected_mutations[line.strip("\n")] = 0
with open("saturational_mutations.tsv") as f:
    for line in f:
        expected_mutations[line.strip("\n")] = 0

barcodes_to_write = []
mutants_written = []
identify_most_abundant_barcodes = []
for key in all_counts_mut_with_counts.keys():
    if len(all_counts_mut_with_counts[key]) > 30:
        identify_most_abundant_barcodes.append(str(key))
        barcode_abundance = []
        mutants_written.append(key)
        for barcode in all_counts_mut_with_counts[key]:
            barcode_abundance.append(all_barcodes_with_counts[barcode])
        all_above = sorted(barcode_abundance)[-30]
        for barcode in all_counts_mut_with_counts[key]:
            if all_barcodes_with_counts[barcode] > all_above-1:
                barcodes_to_write.append(barcode)

print("Mutant variants with more than 30 barcodes: " + str(len(identify_most_abundant_barcodes)) + ", e.g.: " + str(identify_most_abundant_barcodes))
    
unknown_muts_with_counts = 0
unknown_muts_with_counts_more_than_one_barc = {}
known_muts_with_counts = {}
mutantvariants_to_include = []
for mut in all_counts_mut_with_counts.keys():
    if mut in mutants_written:
        if mut not in expected_mutations.keys():
            unknown_muts_with_counts_more_than_one_barc[mut] = 1
            unknown_muts_with_counts += 1
        else:
            known_muts_with_counts[mut] = 1
        mutantvariants_to_include.append(mut)
        continue
    if mut not in expected_mutations.keys():
        unknown_muts_with_counts += 1
        if len(all_counts_mut_with_counts[mut]) > 1:
            barcodes_to_write += all_counts_mut_with_counts[mut]
            unknown_muts_with_counts_more_than_one_barc[mut] = 1
            mutantvariants_to_include.append(str(mut))
    else:
        known_muts_with_counts[mut] = 1
        barcodes_to_write += all_counts_mut_with_counts[mut]
        mutantvariants_to_include.append(str(mut))
        
print("Number mutations not expected with counts above 30: " + str(unknown_muts_with_counts))
print("Number mutations not expected with counts above 30 and more than 1 barcode: " + str(len(unknown_muts_with_counts_more_than_one_barc.keys())))
print(list(unknown_muts_with_counts_more_than_one_barc.keys())[1:10])
print("Number mutations expected with counts above 50: " + str(len(known_muts_with_counts.keys())))
print("That gives in total " + str(len(barcodes_to_write)) + " barcodes to write.")

mut_barcode_dict = {}
with open("output/unfiltered_barcodes.fasta") as f:
    for line in f:
        if line[0] == ">":
            mutant = line.strip("\n")[1:]
        else:
            barcode = line.strip("\n")
            if mutant not in mut_barcode_dict.keys():
                mut_barcode_dict[mutant] = barcode
            else:
                print(mutant + " already encountered - something is wrong...")

with open("output/selected_barcodes_Nextflow.fa", "w") as f:
    for mut in barcodes_to_write:
        f.write(">" + mut + "\n" + mut_barcode_dict[mut] + "\n")

All different mutant variants in all_counts.tsv (regardless of if counts): 19137
Mutant variants with counts > 30: 14617
Number mutant variants with redundancy: 8968
All barcodes with counts > 30: 32549
Mutant variants with more than 30 barcodes: 1, e.g.: ['WT']
Number mutations not expected with counts above 30: 288
Number mutations not expected with counts above 30 and more than 1 barcode: 34
['F405S,Q406R,N407T,STOP407', 'M326F,T347C', 'H141V,M154E,K261F,H265K,V337S,STOP338', 'H141V,M154K,K261E,H265A,Q406D,STOP407', 'H141V,M154S,K261D,H265A,H410T,N412T,STOP412', 'H141L,M154A,K261E,H265A,V337S,H410T,N412T,STOP412', 'M154A,K261E,H265A,H410T,N412T,STOP412', 'H141L,M154K,K261A,V337S,R338A,STOP338', 'H141V,M154S,K261A,H265E,H410T,N412T,STOP412']
Number mutations expected with counts above 50: 14329
That gives in total 31981 barcodes to write.


In [10]:
dict_barcodes = {}
dict_barcodes2 = {}
total = 0
with open("output/selected_barcodes_Nextflow.fa") as f: 
    for line in f:
        if line[0] == ">":
            total += 1
            name = line.strip("\n").strip(">")
            mut = line.strip("\n").strip(">").split("|")[0]
            if mut in dict_barcodes.keys():
                dict_barcodes[mut] += 1
            else:
                dict_barcodes[mut] = 1
        else:
            barcode = line.strip("\n")
            if name in dict_barcodes2.keys():
                print(name + " occurred twice?")
            dict_barcodes2[name] = barcode        
         
print(total)
print(len(list(dict_barcodes2.keys())))
import pandas as pd
df = pd.DataFrame.from_dict(dict_barcodes, orient="index")
df.to_csv("output/selected_barcodes_count.csv")
df2 = pd.DataFrame.from_dict(dict_barcodes2, orient="index")
df2.to_csv("output/selected_barcodes_seq.csv")

31981
31981
